In [121]:
import json

def get_list2scrape(input, compare):
    with open(input, 'r') as f:
        input_list = json.load(f)
    with open(compare, 'r') as f:
        compare_list = json.load(f)

    compare_list = [element['url'] for element in compare_list]

    diff = list(set(input_list) - set(compare_list))

    return input_list, compare_list, diff

In [130]:
inp, comp, diff = get_list2scrape('links.json', 'completed_perfumes.json')

In [131]:
len(inp)

4005

In [132]:
print(f'done: {len(comp)}')

done: 2269


In [133]:
print(f'left {len(diff)}')

left 1736


In [110]:
diff

['https://www.parfumo.de/Parfums/the-dua-brand/Invasion_of_the_Barbers',
 'https://www.parfumo.de/Parfums/Rabanne/phantom-intense',
 'https://www.parfumo.de/Parfums/Lancaster/Le_Parfum_Solaire',
 'https://www.parfumo.de/Parfums/Issey_Miyake/L_Eau_d_Issey_pour_Homme_Eau_d_Ete_Summer_Edition_2014',
 'https://www.parfumo.de/Parfums/der-duft/cinematic',
 'https://www.parfumo.de/Parfums/Roja_Parfums/danger-parfum-cologne',
 'https://www.parfumo.de/Parfums/Salvador_Dali/Silver_Sun',
 'https://www.parfumo.de/Parfums/Roja_Parfums/enigma-parfum-cologne',
 'https://www.parfumo.de/Parfums/Roja_Parfums/51-essence-de-parfum',
 'https://www.parfumo.de/Parfums/Issey_Miyake/L_Eau_d_Issey_Eau_d_Ete_Summer_Edition_2016',
 'https://www.parfumo.de/Parfums/Roja_Parfums/elysium-parfum-cologne-eau-de-parfum',
 'https://www.parfumo.de/Parfums/Guerlain/Muguet_2013',
 'https://www.parfumo.de/Parfums/Issey_Miyake/L_Eau_d_Issey_pour_Homme_Eau_d_Ete_Summer_Edition_2015',
 'https://www.parfumo.de/Parfums/Estee_Laud

In [25]:
file_path = "short_list.json"
new_entries = [{"id": 6, "name": "Charlie"}, {"id": 7, "name": "David"}]

# Open file in append mode and modify JSON manually
with open(file_path, "r+", encoding="utf-8") as file:
    file.seek(0, 2)  # Move to the end of the file
    position = file.tell()
    
    if position > 2:  # If file is not empty, remove the last ']'
        file.seek(position - 2)
        file.truncate()
        file.write(",\n" + json.dumps(new_entries, indent=4)[1:])  # Append new items
    else:
        file.write(json.dumps(new_entries, indent=4)) 

In [ ]:
def save_results(self, results):
    """Save results to JSON file without overwriting existing data"""
    try:
        # Check if the file exists and is not empty
        if os.path.exists(self.output_file) and os.path.getsize(self.output_file) > 0:
            with open(self.output_file, "r+", encoding="utf-8") as f:
                try:
                    existing_data = json.load(f)  # Load existing data
                    if not isinstance(existing_data, list):  # Ensure it's a list
                        raise ValueError("JSON file does not contain a list.")
                except json.JSONDecodeError:
                    existing_data = []  # If corrupted, start fresh

                existing_data.extend(results)  # Append new results

                f.seek(0)  # Move cursor to the beginning
                json.dump(existing_data, f, ensure_ascii=False, indent=2)
                f.truncate()  # Remove leftover data if file shrinks
        else:
            with open(self.output_file, "w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=2)

        print(f"Results saved to {self.output_file}")

    except Exception as e:
        print(f"Error saving results: {e}")

'臠'

In [123]:
dic =  [
                {
                    "r": "0",
                    "v": 0
                },
                {
                    "r": "1",
                    "v": 0
                },
                {
                    "r": "2",
                    "v": 0
                },
                {
                    "r": "3",
                    "v": 3
                },
                {
                    "r": "4",
                    "v": 9
                },
                {
                    "r": "5",
                    "v": 9
                },
                {
                    "r": "6",
                    "v": 18
                },
                {
                    "r": "7",
                    "v": 18
                },
                {
                    "r": "8",
                    "v": 4
                },
                {
                    "r": "9",
                    "v": 0
                },
                {
                    "r": "10",
                    "v": 1
                }
            ]

In [124]:
new_dic = {}
for element in dic:
    print(element)
    new_dic[element["r"]] = element["v"]

{'r': '0', 'v': 0}
{'r': '1', 'v': 0}
{'r': '2', 'v': 0}
{'r': '3', 'v': 3}
{'r': '4', 'v': 9}
{'r': '5', 'v': 9}
{'r': '6', 'v': 18}
{'r': '7', 'v': 18}
{'r': '8', 'v': 4}
{'r': '9', 'v': 0}
{'r': '10', 'v': 1}


In [125]:
new_dic

{'0': 0,
 '1': 0,
 '2': 0,
 '3': 3,
 '4': 9,
 '5': 9,
 '6': 18,
 '7': 18,
 '8': 4,
 '9': 0,
 '10': 1}

In [1]:
import json

In [126]:
with open('completed_perfumes.json', 'r', encoding='utf-8') as f:
    big_dic = json.load(f)


In [127]:
for i in range(1000):
    try:
        # print(big_dic[i]['detailed_ratings']['scent'][0])
        h = 3
    except:
        print(i)
        break

print(big_dic[222]['detailed_ratings'])

{'scent': [{'r': '0', 'v': 0}, {'r': '1', 'v': 0}, {'r': '2', 'v': 0}, {'r': '3', 'v': 0}, {'r': '4', 'v': 3}, {'r': '5', 'v': 12}, {'r': '6', 'v': 28}, {'r': '7', 'v': 76}, {'r': '8', 'v': 53}, {'r': '9', 'v': 14}, {'r': '10', 'v': 9}], 'durability': [{'r': '0', 'v': 0}, {'r': '1', 'v': 0}, {'r': '2', 'v': 1}, {'r': '3', 'v': 1}, {'r': '4', 'v': 4}, {'r': '5', 'v': 10}, {'r': '6', 'v': 34}, {'r': '7', 'v': 85}, {'r': '8', 'v': 24}, {'r': '9', 'v': 4}, {'r': '10', 'v': 2}], 'sillage': [{'r': '0', 'v': 1}, {'r': '1', 'v': 0}, {'r': '2', 'v': 1}, {'r': '3', 'v': 2}, {'r': '4', 'v': 11}, {'r': '5', 'v': 20}, {'r': '6', 'v': 68}, {'r': '7', 'v': 54}, {'r': '8', 'v': 8}, {'r': '9', 'v': 1}, {'r': '10', 'v': 2}], 'bottle': [{'r': '0', 'v': 0}, {'r': '1', 'v': 0}, {'r': '2', 'v': 1}, {'r': '3', 'v': 0}, {'r': '4', 'v': 0}, {'r': '5', 'v': 5}, {'r': '6', 'v': 24}, {'r': '7', 'v': 51}, {'r': '8', 'v': 46}, {'r': '9', 'v': 13}, {'r': '10', 'v': 9}], 'pricing': [{'r': '0', 'v': 0}, {'r': '1', 'v'

In [128]:
for idx, list_element in enumerate(big_dic):

    detailed_ratings = big_dic[idx]['detailed_ratings']

    for typ, rating_list in detailed_ratings.items():
        buf_dic = {}
        if rating_list is not None:
            for rating_element in rating_list:
                buf_dic[rating_element['r']] = rating_element['v']
            detailed_ratings[typ] = buf_dic
        else:
            detailed_ratings[typ] = None 


    big_dic[idx]['detailed_ratings'] = detailed_ratings

In [ ]:
with open('perfumes_complete.json', 'w', encoding='utf-8') as file:
    json.dump(big_dic, file, indent=2)


## Check if proxies work?

In [134]:
import requests

with open('proxies.txt', 'r', encoding='utf-8') as f:
    proxies = [line.strip() for line in f if line.strip()]

In [136]:
import requests

def check_proxy(proxy):
    try:
        response = requests.get("https://httpbin.org/ip", proxies={"http": proxy, "https": proxy}, timeout=5)
        return response.status_code == 200
    except requests.RequestException:
        return False

In [138]:
import requests
from concurrent.futures import ThreadPoolExecutor

def check_proxy(proxy):
    try:
        response = requests.get("https://httpbin.org/ip", proxies={"http": proxy, "https": proxy}, timeout=5)
        if response.status_code == 200:
            print(f'{proxy} is fine')
            return proxy
    except requests.RequestException:
        return None

proxies = [line.strip() for line in open("proxies.txt") if line.strip()]

with ThreadPoolExecutor(max_workers=10) as executor:
    working_proxies = list(filter(None, executor.map(check_proxy, proxies)))

print("Working proxies:", working_proxies)

http://183.240.196.55:38080 is fine
http://125.87.86.178:2324 is fine
http://67.43.236.19:27047 is fine
http://119.96.72.199:59394 is fine
http://98.8.195.160:443 is fine
http://78.101.91.191:8080 is fine
http://38.51.61.53:999 is fine
http://203.142.78.197:8080 is fine
http://38.51.207.98:999 is fine
http://62.33.53.248:3128 is fine
http://158.101.175.124:5566 is fine
http://201.73.228.20:3128 is fine
http://189.240.60.162:9090 is fine
http://45.6.1.97:999 is fine
http://103.163.80.101:8082 is fine
http://103.165.155.68:1111 is fine
http://216.229.112.25:8080 is fine
http://120.133.37.235:1080 is fine
http://36.90.24.5:8080 is fine
http://67.43.228.250:18351 is fine
http://38.51.48.235:999 is fine
http://103.105.54.99:7777 is fine
http://59.63.205.36:655 is fine
http://41.72.213.182:8090 is fine
http://85.132.37.9:1313 is fine
http://186.167.80.236:8090 is fine
http://45.87.68.9:15321 is fine


KeyboardInterrupt: 

In [ ]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def check_proxy(proxy):
    try:
        response = requests.get("https://httpbin.org/ip", proxies={"http": proxy, "https": proxy}, timeout=5)
        if response.status_code == 200:
            return proxy
    except requests.RequestException:
        return None

proxies = [line.strip() for line in open("proxies.txt") if line.strip()]
working_proxies = []

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(check_proxy, proxy): proxy for proxy in proxies}

    for future in tqdm(as_completed(futures), total=len(proxies), desc="Checking Proxies"):
        result = future.result()
        if result:
            working_proxies.append(result)

print("Working proxies:", working_proxies)